<a href="https://colab.research.google.com/github/MatthewMair42/matthewtest/blob/main/MIMI_reg_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIMI Regression Test

Read in the data

In [ ]:
import pandas as pd

# Specify the correct data types for the problematic columns
data = pd.read_csv('/content/mimili_added.csv', dtype={'61': str, '67': str, '69': str, '70': str, '79': str, '85': str, '87': str, '88': str})

data.shape

<ipython-input-38-9cbc42ce9250>:4: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/mimili_added.csv', dtype={'61': str, '67': str, '69': str, '70': str, '79': str, '85': str, '87': str, '88': str})


(662883, 135)

In [ ]:
import statsmodels.formula.api as smf

# Define the regression formula

formula = "combined_T_T_res ~ UN_origin_pop + UN_destination_pop + origin_gdp + destination_gdp + ESTAT_origin_pop_dens + ESTAT_destination_pop_dens + dest_migr_stocks_T + origin_migr_stocks_T + net_migration_origin + net_migration_dest + geodesic_distance_km + common_language + common_religion + cultural_difference + scaled_sci + sci_distance + neighbor_move + year_2000 + year_2001 + year_2002 + year_2003 + year_2004 + year_2005 + year_2006 + year_2007 + year_2008 + year_2009 + year_2010 + year_2011 + year_2012 + year_2013 + year_2014 + year_2015 + year_2016 + year_2017 + year_2018 + year_2019 + year_2020 + year_2021 + year_2022"

# Fit the regression model
try:
    model = smf.ols(formula, data=data).fit()

    # Print the regression results
    print(model.summary())
except KeyError as e:
    print(f"Error: Missing column in the data: {e}")
    print("Please ensure your data file contains the required columns: 'combined_T_T_res', 'origin_gdp', 'origin_pop', 'common_lanugage', and 'common_religion'.")
except Exception as e:  # Catch other potential errors during model fitting
  print(f"An unexpected error occurred during regression: {e}")

                            OLS Regression Results                            
Dep. Variable:       combined_T_T_res   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     202.4
Date:                Thu, 17 Apr 2025   Prob (F-statistic):               0.00
Time:                        20:50:09   Log-Likelihood:                -33218.
No. Observations:                3416   AIC:                         6.649e+04
Df Residuals:                    3387   BIC:                         6.667e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [ ]:
# dataframe and plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# scaling and dataset split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# OLS, Ridge
from sklearn.linear_model import LinearRegression, Lasso
# model evaluation
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
# Select features and outcome
X = data[['UN_origin_pop', 'UN_destination_pop', 'origin_gdp', 'destination_gdp',
          'common_language', 'common_religion', 'scaled_sci']]
y = data['combined_T_T_res']

# Combine and drop rows with any missing values
Xy = pd.concat([X, y], axis=1).dropna()

print(Xy.shape)  # prints (rows, columns)

# Split back
X_clean = Xy.drop(columns='combined_T_T_res')
y_clean = Xy['combined_T_T_res']

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# Now split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_clean, test_size=0.3, random_state=10)

# Final NaN check (just in case)
print(np.isnan(X_train).sum(), np.isnan(y_train).sum())  # Should both be 0


(109733, 8)
0 0


In [ ]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)


Lasso(alpha=0.1)

In [ ]:
y_pred = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


coef = pd.Series(lasso.coef_, index=X.columns)
print(coef)

Mean Squared Error: 871273934.6054765
UN_origin_pop         545.855356
UN_destination_pop    238.272430
origin_gdp             32.290477
destination_gdp       101.611300
common_language       891.131910
common_religion       229.263689
scaled_sci            141.566709
dtype: float64


In [ ]:
from sklearn.linear_model import Lasso
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

formula = 'combined_T_T_res ~ UN_origin_pop + UN_destination_pop + origin_gdp + destination_gdp + common_language + common_religion + scaled_sci'


# OLS (using sklearn for clean comparison to LASSO)
ols = sm.OLS(y_train, sm.add_constant(X_train)).fit()
y_pred_ols = ols.predict(sm.add_constant(X_test))

# LASSO
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf

# --- Set your outcome and features ---
outcome_var = 'combined_T_T_res'
features = [
    'UN_origin_pop', 'UN_destination_pop', 'origin_gdp', 'destination_gdp',
    'common_language', 'common_religion', 'scaled_sci'
]

# --- Prepare your formula ---
formula = outcome_var + ' ~ ' + ' + '.join(features)

# --- Drop NA from both X and y safely ---
Xy = data[features + [outcome_var]].dropna()
X = Xy[features]
y = Xy[outcome_var]

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- Fit OLS using formula-based API ---
train_df = X_train.copy()
train_df[outcome_var] = y_train
model_ols = smf.ols(formula=formula, data=train_df).fit()

# --- Predict on test set using OLS ---
test_df = X_test.copy()
test_df[outcome_var] = y_test
y_pred_ols = model_ols.predict(test_df)

# --- Standardize X for LASSO ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Fit LASSO ---
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
y_pred_lasso = lasso.predict(X_test_scaled)

# --- Calculate RMSE for both models ---
rmse_ols = np.sqrt(mean_squared_error(y_test, y_pred_ols))
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

print(f"OLS RMSE: {rmse_ols:.4f}")
print(f"LASSO RMSE: {rmse_lasso:.4f}")


OLS RMSE: 15717.0926
LASSO RMSE: 15717.0939


In [ ]:
# Get OLS coefficients (drop intercept if present)
ols_coefs = model_ols.params.drop('Intercept', errors='ignore')

# Get LASSO coefficients and match with feature names
lasso_coefs = pd.Series(lasso.coef_, index=features)

# Combine into one DataFrame
coef_comparison = pd.DataFrame({
    'OLS Coefficient': ols_coefs,
    'LASSO Coefficient': lasso_coefs
})

print(coef_comparison)


                    OLS Coefficient  LASSO Coefficient
UN_origin_pop              0.000005         480.652721
UN_destination_pop         0.000003         242.055097
origin_gdp                 0.000820          16.231192
destination_gdp            0.004709          91.074477
common_language         2661.107112         941.060894
common_religion          497.263498         241.838809
scaled_sci                 0.000058         240.857947
